# Vaje 3: Merjenje napovedne napake modela

## Naloga 1: Prečno preverjanje in stabilnost modela

In [1]:
import numpy as np

# Preberemo podatke shranje v numpy formatu (s funkcijo numpy.save(pot, array))
data = np.load("../Podatki/vaje3_1.npy")
X = data[:, :-1]
y = data[:, -1]

1.a: Preveri (povprečno) točnost linearne regresije s petkratnim prečnim preverjanjem. Kako stabilen je model oz. kakšna je varianca dobljenih napak?

<details>
  <summary>Namig:</summary>

  *Pomagaj si z [objektom sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) in njegovo metodo split(X)*.
   
</details>

In [2]:
# Importamo KFold objekt, linearno regresijo in metriko MSE
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = KFold(n_splits=5)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(X)):
    # Z array-em train_index izberemo podatke za učno množico
    x_train = X[train_index, :]
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    x_test = X[test_index, :]
    y_test = y[test_index]
    # Natreniramo model linearne regresije
    model = LinearRegression().fit(x_train, y_train)
    # Napovemo ciljno vrednost testnih podatkov
    y_pred = model.predict(x_test)
    # Izračunamo RMSE testnih podatkov
    error = np.sqrt(mean_squared_error(y_test, y_pred))
    # Napako shranimo v list errors
    errors.append(error)
    
    print(f"Fold {i}: RMSE {error}")
    
# Izpišemo povprečno vrednost in varianco napak
print(f"Mean RMSE: {np.mean(errors)}")
print(f"Variance of RMSE: {np.var(errors)}")

Fold 0: RMSE 0.18960165232406972
Fold 1: RMSE 0.1863886412398821
Fold 2: RMSE 0.19605889866247667
Fold 3: RMSE 0.17717484439089284
Fold 4: RMSE 0.17002566428275065
Mean RMSE: 0.1838499401800144
Variance of RMSE: 8.485067415271e-05


1.b: S prečnim preverjanjem sestavi pet modelov linearne regresije ter si (v matriko velikosti 5x4) shrani njihove začetne vrednosti in koeficiente. Se istoležni koeficienti v različnih vzorcih (Foldih) razlikujejo? Za koliko?

In [4]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = KFold(n_splits=5)
# Inicializiramo array, v katerega bomo shranjevali koeficiente. Ta ima 5 vrstic (saj delamo petkratno prečno preverjanje) in 4 stolpce (začetna vrednost + 3 koeficienti)
coeffs = np.zeros((5, 4))
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
for i, (train_index, test_index) in enumerate(kfold.split(X)):
    # Izberemo učno
    x_train = X[train_index, :]
    y_train = y[train_index]
    # Natreniramo model linearne regresije
    model = LinearRegression().fit(x_train, y_train)
    # Začetno vrednost shranimo v prvi stolpec
    coeffs[i, 0] = model.intercept_
    # Koeficiente shranimo v stolpce 1, 2 in 3
    coeffs[i, 1:] = model.coef_

# Izpišemo koeficiente
print("Coefficients:")
print(coeffs)
# Izpišemo variance, axis=0 poskrbi, da varianco zračunamo glede na stolpce (po 0-ti dimenziji)
print(f"Variance: {np.var(coeffs, axis=0)}")
# Izpišemo povprečno vrednost varianc
print(f"Mean variance: {np.mean(np.var(coeffs, axis=0))}")

Coefficients:
[[-0.18415939  1.50743854  2.43282336  1.47957335]
 [-0.19639105  1.52647655  2.4416169   1.47915122]
 [-0.16964682  1.51169294  2.4205532   1.46727743]
 [-0.15786318  1.48666751  2.43633069  1.47178237]
 [-0.18573746  1.49966098  2.42805102  1.50079499]]
Variance: [1.81683894e-04 1.73389256e-04 5.16926167e-05 1.32464774e-04]
Mean variance: 0.000134807634997536


1.c: Podatkom dodaj spremenljivke drugega reda ($x_1^2$, $x_1\cdot x_2$, $x_1\cdot x_3$, $x_2^2$, ...). Stolpce lahko združiš s funkcijo numpy.concatenate(seznam stolpcev, axis=1). 

In [5]:
# Definiramo list, ki bo vseboval stolpce in vanj damo začetne podatke
columns = [X]
# Gremo čez vse kombinacije stolpcev (00,01,02,11,12,22)
for i in range(3):
    for j in range(i, 3):
        # Izračunamo novo spremenljivko x_i*x_j
        # X[:, i]*X[:, j] nam vrne 1D matriko, želimo pa 2D matriko. 1D matriko A velikosti n lahko v 2D matriko velikosti nx1 spremenimo z ukazom A[:, None].
        # 1D matiko A velikosti n, bi lahko v 2D matiko velikosti 1xn spremenili z ukazom A[None, :]
        x_new = (X[:, i]*X[:, j])[:, None]
        # Nov stolpec dodamo v list columns
        columns.append(x_new)

# Stolpce združimo v 2D matiko z ukazom numpy.concatenate(). Z axis=1 povemo, da hočemo združit glede na 1-vo dimenzijo.
X2 = np.concatenate(columns, axis=1)
        

1.d: Preveri točnost linearne regresije na podatkih iz naloge 1.c s petkratnim prečnim preverjanjem. So koeficienti modela bolj ali manj razlikujejo med različnimi vzorci?

In [6]:
kfold = KFold(n_splits=5)
coeffs = np.zeros((5, X2.shape[1]+1))
errors = []

for i, (train_idx, test_idx) in enumerate(kfold.split(X2)):
    x_train = X2[train_idx, :]
    y_train = y[train_idx]
    x_test = X2[test_idx, :]
    y_test = y[test_idx]
    model = LinearRegression().fit(x_train, y_train)
    coeffs[i, 0] = model.intercept_
    coeffs[i, 1:] = model.coef_

    y_pred = model.predict(x_test)
    error = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"Fold {i} RMSE {error}")
    errors.append(error)

print(f"Mean RMSE {np.mean(errors)}")
print(f"RMSE variance {np.var(errors)}")
print(f"Coefficient mean: {np.mean(coeffs, axis=0)}")
print(f"Variance of coefficients: {np.var(coeffs, axis=0)}")

Fold 0 RMSE 0.10714941463868283
Fold 1 RMSE 0.1171339110187921
Fold 2 RMSE 0.1105842934064686
Fold 3 RMSE 0.10727072639075325
Fold 4 RMSE 0.11280266577829533
Mean RMSE 0.11098820224659842
RMSE variance 1.3956214721817903e-05
Coefficient mean: [-0.16003666  1.81634349  2.65368046  1.64783266 -0.76940341  0.9713595
 -0.0338234  -1.05901487  0.67275282 -0.48365751]
Variance of coefficients: [0.00016214 0.00103623 0.00027615 0.0002361  0.00021888 0.00046882
 0.00017924 0.0005724  0.00036726 0.00037009]


## Naloga 2:  Stratificirano vzorčenje

In [7]:
# Preberemo podatke iz datoteke vaje3_2.npz. Podatke x, y lahko shranimo v datoteko s končnico npz z uporabo funkcije numpy.savez(pot, x=x, y=y)
data = np.load("../Podatki/vaje3_2.npz")
# Podatke shranimo v spremenljivko x
x = data["x"]
# Ciljne vrednosti shranimo v spremenljivko y
y = data["y"]

2.a: Preveri točnost logistične regresije s petkratnim prečnim preverjanjem. Izpiši točnost modela glede na metriko "klasifikacijska točnost" (accuracy) v vsakem vzorcu. Opaziš kaj nenavadnega?

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = KFold(n_splits=5)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(x)):
    # Z array-em train_index izberemo podatke za učno množico
    x_train = x[train_index, :]
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    x_test = x[test_index, :]
    y_test = y[test_index]
    # Natreniramo model linearne regresije
    model = LogisticRegression().fit(x_train, y_train)
    # Napovemo ciljno vrednost testnih podatkov
    y_pred = model.predict(x_test)
    # Izračunamo RMSE testnih podatkov
    error = accuracy_score(y_test, y_pred)
    # Napako shranimo v list errors
    errors.append(error)
    
    print(f"Fold {i}: RMSE {error}")
    
# Izpišemo povprečno vrednost in varianco napak
print(f"Mean accuracy: {np.mean(errors)}")


Fold 0: RMSE 1.0
Fold 1: RMSE 1.0
Fold 2: RMSE 1.0
Fold 3: RMSE 0.66
Fold 4: RMSE 0.0
Mean accuracy: 0.732


2.b: Za vsak vzorec podatkov izpiši število pozitivnih in negativnih vrednosti ciljne spremenljivke v učni in testni množici. Zakaj se je točnost modela v nalogi 2.a tako razlikovala med različnimi vzorci?

In [10]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = KFold(n_splits=5)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(X)):
    # Z array-em train_index izberemo podatke za učno množico
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    y_test = y[test_index]
    print(f"Fold {i}")
    print(f"Train set: {y_train.sum()} positive, {y_train.shape[0] - y_train.sum()} negative")
    print(f"Test set: {y_test.sum()} positive, {y_test.shape[0] - y_test.sum()} negative")
    print()

Fold 0
Train set: 201.0 positive, 599.0 negative
Test set: 0.0 positive, 200.0 negative

Fold 1
Train set: 201.0 positive, 599.0 negative
Test set: 0.0 positive, 200.0 negative

Fold 2
Train set: 201.0 positive, 599.0 negative
Test set: 0.0 positive, 200.0 negative

Fold 3
Train set: 200.0 positive, 600.0 negative
Test set: 1.0 positive, 199.0 negative

Fold 4
Train set: 1.0 positive, 799.0 negative
Test set: 200.0 positive, 0.0 negative



2.c: Če je distribucija ciljne spremenljivke v vzorcih učne in testne množice podobna originalni distribuciji ciljne spremenljivke, takemu vzorčenju rečemo stratificirano vzorčenje. Sestavi stratificirane vzorce za petkratno prečno preverjanje in preveri koliko pozitivnih in koliko negativnih primerov vsebuje učna in testna množica.


<details>
  <summary>Namig:</summary>

  *Pomagaj si z [objektom sklearn.model_selection.StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) in njegovo metodo split(x, y)*.
   
</details>

In [11]:
from sklearn.model_selection import StratifiedKFold

In [12]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = StratifiedKFold(n_splits=5)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    # Z array-em train_index izberemo podatke za učno množico
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    y_test = y[test_index]
    print(f"Fold {i}")
    print(f"Train set: {y_train.sum()} positive, {y_train.shape[0] - y_train.sum()} negative")
    print(f"Test set: {y_test.sum()} positive, {y_test.shape[0] - y_test.sum()} negative")
    print()

Fold 0
Train set: 161.0 positive, 639.0 negative
Test set: 40.0 positive, 160.0 negative

Fold 1
Train set: 161.0 positive, 639.0 negative
Test set: 40.0 positive, 160.0 negative

Fold 2
Train set: 161.0 positive, 639.0 negative
Test set: 40.0 positive, 160.0 negative

Fold 3
Train set: 161.0 positive, 639.0 negative
Test set: 40.0 positive, 160.0 negative

Fold 4
Train set: 160.0 positive, 640.0 negative
Test set: 41.0 positive, 159.0 negative



2.d: Preveri točnost logistične regresije s petkratnim prečnim preverjanjem na vzorcih, ki jih dobiš s stratificiranim vzorčenjem. So dobljeni modeli bolj stabilni? Si pričakoval/a da bodo dobljeni rezultati bolj stabilni?

In [13]:
# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = StratifiedKFold(n_splits=5)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    # Z array-em train_index izberemo podatke za učno množico
    x_train = x[train_index, :]
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    x_test = x[test_index, :]
    y_test = y[test_index]
    # Natreniramo model linearne regresije
    model = LogisticRegression().fit(x_train, y_train)
    # Napovemo ciljno vrednost testnih podatkov
    y_pred = model.predict(x_test)
    # Izračunamo RMSE testnih podatkov
    error = accuracy_score(y_test, y_pred)
    # Napako shranimo v list errors
    errors.append(error)
    
    print(f"Fold {i}: RMSE {error}")
    
# Izpišemo povprečno vrednost in varianco napak
print(f"Mean accuracy: {np.mean(errors)}")

Fold 0: RMSE 0.835
Fold 1: RMSE 0.94
Fold 2: RMSE 0.99
Fold 3: RMSE 1.0
Fold 4: RMSE 0.785
Mean accuracy: 0.9099999999999999


2.e: Pred stratificiranim vzorčenjem podatke še premešaj. To lahko narediš tako, da objektu StratifiedKFold dodaš parameter shuffle=True. So sedaj rezultati bolj stabilni? Kaj se zgodi, če kodo poženeš večkrat?

In [14]:
# Nastavimo "seme" generatorja naključnih števil
np.random.seed(18)

# Inicializiramo KFold objekt in določimo število vzorcev, ki jih bo naredil (5)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
# Inicializiramo list v katerega bomo shranjevali napake
errors = []
# Dobimo indekse učnih in testnih podatkov z metodo kfold.split(množica)
# Z enumerate indeksiramo fold, v katerem smo
for i, (train_index, test_index) in enumerate(kfold.split(x, y)):
    # Z array-em train_index izberemo podatke za učno množico
    x_train = x[train_index, :]
    y_train = y[train_index]
    # Z array-em test_index izberemo podatke za testno množico
    x_test = x[test_index, :]
    y_test = y[test_index]
    # Natreniramo model linearne regresije
    model = LogisticRegression().fit(x_train, y_train)
    # Napovemo ciljno vrednost testnih podatkov
    y_pred = model.predict(x_test)
    # Izračunamo RMSE testnih podatkov
    error = accuracy_score(y_test, y_pred)
    # Napako shranimo v list errors
    errors.append(error)
    
    print(f"Fold {i}: RMSE {error}")
    
# Izpišemo povprečno vrednost in varianco napak
print(f"Mean accuracy: {np.mean(errors)}")

Fold 0: RMSE 0.965
Fold 1: RMSE 0.975
Fold 2: RMSE 0.95
Fold 3: RMSE 0.95
Fold 4: RMSE 0.965
Mean accuracy: 0.961


Opomba: Parameter shuffle naredi stratificirano vzorčenje stohastično s pomočjo generatorja naključnih števil. Z metodo numpy.random.seed(celo število) lahko poskrbimo, da bo generator vedno vračal ista naključna števil in bodo poslednično naši eksperimenti ponovljivi.

Pozor: Vpliv na točnost modela je le posledica random seed-a. V publikacijah se uporablja le za ponovljivost eksperimentov (in ne kot: Najboljši rezultat dobimo pri random seed-u 18)